In [27]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from datetime import timedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

In [28]:
route = pd.read_csv("/content/drive/MyDrive/Dataset/upa/preprocess/C-001_route2.csv",index_col=0,parse_dates=["Date from","Date to"])
env = pd.read_csv("/content/drive/MyDrive/Dataset/upa/C-001_env.csv",parse_dates=["Date"])
route = route.iloc[:159,:]
env.drop("No.",axis=1,inplace=True)
env.set_index("Date",inplace=True)

In [29]:
route.head()

,Date from,Date to,days,Port from,Port to,Distance,T from,T to,Loaing Status,Mega-Ton-Mile,Cargo Carried,DFOC,Wave (per Distance),Acc. Fatigue Inex,Wind (per Distance),Avg. Wave Height,Avg. Speed,Avg. Win Speed
0,2018-01-01 21:30:00,2018-01-03 10:19:00,1.5,KR0016,unkown,170,13.0,13.0,NaN,0.000,0.0,0.0,1.9,41.3,5.9,2.1,14.5,5.1
1,2018-01-03 10:19:00,2018-01-05 11:15:00,2.0,unkown,CN0603,782,13.0,12.0,NaN,0.000,0.0,0.0,1.5,30.9,3.6,1.5,5.0,3.6
2,2018-01-05 11:15:00,2018-01-06 15:34:00,1.2,CN0603,KR0080,1380,12.0,16.0,NaN,656.393,71570.0,0.0,1.4,19.5,1.9,1.5,10.5,4.6
3,2018-01-06 15:34:00,2018-01-09 04:43:00,2.5,KR0080,CN0200,1048,16.0,12.0,NaN,0.000,0.0,0.0,2.0,98.3,7.3,2.1,2.7,6.4
4,2018-01-09 04:43:00,2018-01-10 16:55:00,1.5,CN0200,CN0565,976,12.0,12.0,NaN,0.000,0.0,0.0,2.2,69.8,3.8,2.1,18.0,3.7


In [30]:
date = env.index.to_list()
result_list = []

for i in date:

  try:

    df = route[(route["Date from"] <= i) & (route["Date to"] >= i)]
    df = df.iloc[0,:].values
    result_list.append(df)

  except:

    empty_list = np.empty((1,15))
    empty_list[:] = np.nan
    result_list.append(empty_list[0].tolist())

df = pd.DataFrame(result_list,columns=route.columns,index=env.index)

In [31]:
df

,Date from,Date to,days,Port from,Port to,Distance,T from,T to,Loaing Status,Mega-Ton-Mile,Cargo Carried,DFOC,Wave (per Distance),Acc. Fatigue Inex,Wind (per Distance),Avg. Wave Height,Avg. Speed,Avg. Win Speed
Date,,,,,,,,,,,,,,,,,,
2018-01-01 06:00:00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 12:00:00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 18:00:00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02 00:00:00,2018-01-01 21:30:00,2018-01-03 10:19:00,1.5,KR0016,unkown,170.0,13.0,13.0,NaN,0.0,0.0,0.0,1.9,41.3,5.9,2.1,14.5,5.1
2018-01-03 06:00:00,2018-01-01 21:30:00,2018-01-03 10:19:00,1.5,KR0016,unkown,170.0,13.0,13.0,NaN,0.0,0.0,0.0,1.9,41.3,5.9,2.1,14.5,5.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27 18:00:00,2018-12-27 02:22:00,2018-12-29 02:14:00,2.0,RP4610,I0238,1623.0,10.0,10.0,NaN,0.0,0.0,0.0,1.5,76.9,5.2,1.5,18.2,4.6
2018-12-28 00:00:00,2018-12-27 02:22:00,2018-12-29 02:14:00,2.0,RP4610,I0238,1623.0,10.0,10.0,NaN,0.0,0.0,0.0,1.5,76.9,5.2,1.5,18.2,4.6
2018-12-28 06:00:00,2018-12-27 02:22:00,2018-12-29 02:14:00,2.0,RP4610,I0238,1623.0,10.0,10.0,NaN,0.0,0.0,0.0,1.5,76.9,5.2,1.5,18.2,4.6


In [32]:
env = pd.concat([env,df],axis=1)

In [34]:
env.isnull().sum()

Vessel Latitude           0
Vessel Longitude          0
Vessel Speed (Kts)        0
Vessel Heading            0
Wave Height (m)           0
Wave Period (s)           0
Wind U (m/s)              0
Wind V (m/s)              0
Date from               156
Date to                 156
days                    156
Port from               156
Port to                 156
Distance                156
T from                  156
T to                    156
Loaing Status          1267
Mega-Ton-Mile           156
Cargo Carried           156
DFOC                    156
Wave (per Distance)     156
Acc. Fatigue Inex       156
Wind (per Distance)     156
Avg. Wave Height        156
Avg. Speed              156
Avg. Win Speed          156
dtype: int64

In [36]:
env.drop("Loaing Status",axis=1,inplace=True)

# Port from

In [37]:
train_df = env[env["Port from"] != "unkown"]
val_df = env[env["Port from"] == "unkown"]

train_df.dropna(inplace=True)
val_df.dropna(inplace=True)

date_from = val_df["Date from"].to_frame(name="Date from")
date_to = val_df["Date to"].to_frame(name="Date to")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [38]:
train_df.drop(["Port to","Date from","Date to"],axis=1,inplace=True)
val_df.drop(["Port to","Date from","Date to"],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
X_train = train_df.drop("Port from",axis=1)
y_train = train_df.loc[:,"Port from"].to_frame(name="Port from")
X_val = val_df.drop("Port from",axis=1)
y_val = val_df.loc[:,"Port from"].to_frame(name="Port from")

In [18]:
scaler = MinMaxScaler()
encoder = LabelEncoder()

In [40]:
X_train = pd.DataFrame(scaler.fit_transform(X_train),index=X_train.index,columns=X_train.columns)
y_train = pd.DataFrame(encoder.fit_transform(y_train),index=y_train.index,columns=y_train.columns)
X_val = pd.DataFrame(scaler.fit_transform(X_val),index=X_val.index,columns=X_val.columns)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [41]:
 X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size=0.5)

In [42]:
RFC = RandomForestClassifier()

RFC.fit(X_train,y_train)
pred = RFC.predict(X_test)
accuracy = accuracy_score(y_test,pred)
f1 = f1_score(y_test,pred,average="micro")
print(accuracy)
print(f1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.986328125
0.986328125


In [43]:
val_pred = encoder.inverse_transform(RFC.predict(X_val))

In [44]:
port_from_result = pd.DataFrame(val_pred,index=val_df.index,columns=["Port from"])
port_from_result = pd.concat([port_from_result,date_from,date_to],axis=1)

In [45]:
port_from_result

,Port from,Date from,Date to
Date,,,
2018-01-03 12:00:00,I0238,2018-01-03 10:19:00,2018-01-05 11:15:00
2018-01-04 00:00:00,I0238,2018-01-03 10:19:00,2018-01-05 11:15:00
2018-01-04 06:00:00,I0238,2018-01-03 10:19:00,2018-01-05 11:15:00
2018-01-04 12:00:00,I0238,2018-01-03 10:19:00,2018-01-05 11:15:00
2018-01-04 18:00:00,I0238,2018-01-03 10:19:00,2018-01-05 11:15:00
...,...,...,...
2018-12-18 12:00:00,I0238,2018-12-17 17:20:00,2018-12-19 23:54:00
2018-12-19 00:00:00,I0238,2018-12-17 17:20:00,2018-12-19 23:54:00
2018-12-19 06:00:00,I0238,2018-12-17 17:20:00,2018-12-19 23:54:00


In [ ]:
port_from_result.to_csv("/content/drive/MyDrive/Dataset/upa/result/port_from_result.csv")

# Port to

In [46]:
train_df = env[env["Port to"] != "unkown"]
val_df = env[env["Port to"] == "unkown"]

train_df.dropna(inplace=True)
val_df.dropna(inplace=True)

date_from = val_df["Date from"].to_frame(name="Date from")
date_to = val_df["Date to"].to_frame(name="Date to")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [47]:
train_df.drop(["Port from","Date from","Date to"],axis=1,inplace=True)
val_df.drop(["Port from","Date from","Date to"],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [49]:
train_df

,Vessel Latitude,Vessel Longitude,Vessel Speed (Kts),Vessel Heading,Wave Height (m),Wave Period (s),Wind U (m/s),Wind V (m/s),days,Port to,Distance,T from,T to,Mega-Ton-Mile,Cargo Carried,DFOC,Wave (per Distance),Acc. Fatigue Inex,Wind (per Distance),Avg. Wave Height,Avg. Speed,Avg. Win Speed
Date,,,,,,,,,,,,,,,,,,,,,,
2018-01-03 12:00:00,34.14310,127.9090,12.2117,182,1.72568,8.44688,-5.06831,-2.873990,2.0,CN0603,782.0,13.0,12.0,0.0,0.0,0.0,1.5,30.9,3.6,1.5,5.0,3.6
2018-01-04 00:00:00,31.20940,124.7170,10.4558,261,1.41658,8.31981,-4.03375,-2.668770,2.0,CN0603,782.0,13.0,12.0,0.0,0.0,0.0,1.5,30.9,3.6,1.5,5.0,3.6
2018-01-04 06:00:00,31.95590,125.5790,10.4558,261,1.34916,7.60229,-4.51793,-2.872190,2.0,CN0603,782.0,13.0,12.0,0.0,0.0,0.0,1.5,30.9,3.6,1.5,5.0,3.6
2018-01-04 12:00:00,32.70240,126.4400,10.4558,261,1.66967,7.58741,-4.06246,-0.836129,2.0,CN0603,782.0,13.0,12.0,0.0,0.0,0.0,1.5,30.9,3.6,1.5,5.0,3.6
2018-01-04 18:00:00,33.44890,127.3020,10.4558,261,1.97152,8.06035,-1.45173,-1.886560,2.0,CN0603,782.0,13.0,12.0,0.0,0.0,0.0,1.5,30.9,3.6,1.5,5.0,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27 18:00:00,5.66443,85.1721,18.2422,63,1.46807,9.59429,-13.16300,8.434380,2.0,I0238,1623.0,10.0,10.0,0.0,0.0,0.0,1.5,76.9,5.2,1.5,18.2,4.6
2018-12-28 00:00:00,5.71861,86.5901,18.4793,63,1.45147,9.47140,-18.34170,8.811440,2.0,I0238,1623.0,10.0,10.0,0.0,0.0,0.0,1.5,76.9,5.2,1.5,18.2,4.6
2018-12-28 06:00:00,5.85843,90.4939,17.8055,63,1.39000,8.78855,-7.17954,18.288500,2.0,I0238,1623.0,10.0,10.0,0.0,0.0,0.0,1.5,76.9,5.2,1.5,18.2,4.6


In [50]:
X_train = train_df.drop("Port to",axis=1)
y_train = train_df.loc[:,"Port to"].to_frame(name="Port to")
X_val = val_df.drop("Port to",axis=1)
y_val = val_df.loc[:,"Port to"].to_frame(name="Port from")

In [51]:
X_train = pd.DataFrame(scaler.fit_transform(X_train),index=X_train.index,columns=X_train.columns)
y_train = pd.DataFrame(encoder.fit_transform(y_train),index=y_train.index,columns=y_train.columns)
X_val = pd.DataFrame(scaler.fit_transform(X_val),index=X_val.index,columns=X_val.columns)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [52]:
 X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size=0.5)

In [53]:
RFC = RandomForestClassifier()

RFC.fit(X_train,y_train)
pred = RFC.predict(X_test)
accuracy = accuracy_score(y_test,pred)
f1 = f1_score(y_test,pred,average="micro")
print(accuracy)
print(f1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.9652173913043478
0.9652173913043478


In [54]:
val_pred = encoder.inverse_transform(RFC.predict(X_val))

In [55]:
port_to_result = pd.DataFrame(val_pred,index=val_df.index,columns=["Port to"])
port_to_result = pd.concat([port_to_result,date_from,date_to],axis=1)

In [65]:
result = port_to_result.groupby("Date to")["Port to"].value_counts().to_frame(name="Count")

In [66]:
result.to_csv("/content/drive/MyDrive/Dataset/upa/result/C-001_port_to_result.csv")